In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to("cuda")
embeddings = model.encode(sentences)
print(embeddings)


C:\Users\Umut\anaconda3\envs\inzva\Lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


[[ 0.02250258 -0.07829175 -0.02303072 ... -0.00827933  0.02652685
  -0.00201897]
 [ 0.04170234  0.00109742 -0.01553421 ... -0.02181628 -0.06359358
  -0.00875286]]


In [4]:
import pandas as pd

In [5]:
train=pd.read_csv("Data/train.csv")
test=pd.read_csv("Data/test.csv")


In [6]:
len(train)

7613

In [7]:
len(train["keyword"].unique())

222

In [4]:
train.drop(["keyword",'location',"id"],axis=1,inplace=True)
test.drop(["keyword",'location'],axis=1,inplace=True)

In [5]:
train["encoded"]=train["text"].apply(model.encode)

In [6]:
test["encoded"]=test["text"].apply(model.encode)

In [7]:
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np

X=np.array(pd.DataFrame(train["encoded"].to_list(),columns=["x"+str(i) for i in range(768)]))
Y=np.array(train["target"])

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.05)



In [8]:
len(x_train)


7232

In [9]:
x_train

array([[-0.06475862, -0.04728267, -0.00494481, ...,  0.02159298,
        -0.06191199, -0.01174091],
       [-0.01076619,  0.00460944,  0.00558986, ...,  0.01231898,
        -0.05049714, -0.00856717],
       [ 0.00972797, -0.03170099, -0.00049102, ...,  0.08379747,
        -0.01493549, -0.01204772],
       ...,
       [ 0.00075406,  0.09063631,  0.00186593, ..., -0.02349607,
        -0.02578635, -0.00114799],
       [-0.0008723 ,  0.08929235,  0.01109358, ...,  0.02809023,
        -0.05283959, -0.0248389 ],
       [-0.01156183,  0.0747918 , -0.01525769, ..., -0.03582247,
         0.02373038, -0.03704655]], dtype=float32)

In [10]:
len(y_train)

7232

In [11]:
adaboost=AdaBoostClassifier()

adaboost.fit(x_train,y_train)

C:\Users\Umut\anaconda3\envs\inzva\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier()

In [12]:
pred=adaboost.predict(x_test)

In [13]:
from sklearn.metrics import confusion_matrix,accuracy_score

cm=confusion_matrix(y_test,pred)

In [14]:
accuracy_score(y_test,pred)

0.7821522309711286

In [15]:
cm


array([[175,  46],
       [ 37, 123]])

In [16]:
X_sub=np.array(pd.DataFrame(test["encoded"].to_list(),columns=["x"+str(i) for i in range(768)]))

In [17]:
result_pred=adaboost.predict(X_sub)

In [18]:
result_pred

array([0, 1, 1, ..., 1, 1, 1])

In [19]:
result=pd.DataFrame.from_dict({"id":test["id"],
                        "target":result_pred})

In [20]:
result.to_csv("result.csv",index=False)

# NN

In [21]:
from Model import FFN
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader
import torch
model=FFN(768,1)

optim=Adam(model.parameters())
loss_function=BCEWithLogitsLoss()



In [22]:

x_train =torch.Tensor(x_train)
y_train =torch.Tensor(y_train)
x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

train_dataset=TensorDataset(x_train,y_train)
test_dataset=TensorDataset(x_test,y_test)

In [27]:
train_loader=DataLoader(train_dataset,batch_size=32)
test_loader=DataLoader(test_dataset)

In [24]:
from tqdm import tqdm

In [37]:

EPOCH=100

training_loss=[]
test_loss=[]
for epoch in range(EPOCH):
    running_loss=0
    
    
    for idx,data in enumerate(tqdm(train_loader)):
        
        inputs,labels=data
        labels=labels.unsqueeze(-1)
        optim.zero_grad()
        
        outputs=model(inputs)
        
        loss=loss_function(outputs,labels)
        loss.backward()
        
        optim.step()
        
        running_loss+=(loss.item())
    
    training_loss.append((running_loss/(idx+1)))    
    
    with torch.no_grad():
        
        pred=model(x_test)
        loss=loss_function(pred,y_test.unsqueeze(-1))
        
        test_loss.append(loss.item())
        
    print("Training Loss",training_loss[epoch])
    print("Test Loss",test_loss[epoch])
        
    

    
    
    

100%|██████████| 226/226 [00:00<00:00, 404.60it/s]


Training Loss 0.1625909998650308
Test Loss 0.6720555424690247


100%|██████████| 226/226 [00:00<00:00, 390.66it/s]


Training Loss 0.13419777091460683
Test Loss 0.7441050410270691


100%|██████████| 226/226 [00:00<00:00, 393.95it/s]


...

Training Loss 0.025145818451891202
Test Loss 1.8539659976959229


100%|██████████| 226/226 [00:00<00:00, 393.46it/s]


Training Loss 0.026793794427502474
Test Loss 1.9031404256820679


100%|██████████| 226/226 [00:00<00:00, 397.41it/s]

Training Loss 0.042170355258590905
Test Loss 1.7880957126617432


In [48]:

res=torch.nn.functional.sigmoid(model(torch.tensor(X_sub)))

In [50]:
res=[1 if i>0.5 else 0 for i in res]

In [53]:

result2=pd.DataFrame.from_dict({"id":test["id"],
                                "target":res})

In [56]:
result2.to_csv("resultnn.csv",index=False)